In [511]:
import pandas as pd
import csv
## should add info about line up being shaded left or right over center as well as jet/bear with 2 TE's left/right

In [495]:
fronts = pd.read_csv("front finder sheet.xlsx - shaped (1).csv")

In [513]:
## Takes any cell and creates a seperates each position into a list of positions
def positions_list(cell):
    positions = cell.split("; ")
    return positions

In [514]:
## Tells me how many TEs are present on offensive line
def tight_ends_present(cell):
    cell = positions_list(cell)
    TE_count = 0
    for code in cell:
        if "TE" in code:
            TE_count+=1
    return TE_count

In [515]:
## If there are tight ends present this function will tell me how many are on left/right side of the line

def tight_ends_number_and_location(arg):
    TE_left = 0
    TE_right = 0
    positions = positions_list(arg)
    for pos in positions:
        if "TE-L" in pos:
            TE_left +=1
        elif "TE-oL" in pos:
            TE_left +=1
        elif "TE-iL" in pos:
            TE_left +=1
        elif "TE-R" in pos:
            TE_right +=1
        elif "TE-oR" in pos:
            TE_right +=1
        elif "TE-iR" in pos:
            TE_right +=1
            
    return TE_left, TE_right            

In [516]:
## Will show a player's position on the line or depth off the line of scrimmage in a list of lists
def defensive_decoder(arg):
    positions = positions_list(arg)
    tuples = []
    for pos in positions:
        temp_list = pos.split(" ")
        tuples.append(temp_list)
    i = 0
    while i<len(tuples):
        num = tuples[i][1]
        num = int(num[1:-1])
        tuples[i][1] = num
        i+=1
    return tuples
#defensive_decoder(fronts.iloc[2,0])

In [440]:
## Will remover any player from a list of positions if they are playing off the ball
def off_ball_remover(arg1, arg2):
    all_positions = defensive_decoder(arg1)
    off_ball = defensive_decoder(arg2)
    for pos in off_ball:
        if pos[1]>1:
            for item in all_positions:
                if pos[0] == item[0]:
                    all_positions.remove(item)
    
    return all_positions

In [441]:
## This combines the two above functions and will be used to remove players who are 
## playing off the ball from the list of DEF FRONT players
def defensive_locations(arg1, arg2, arg3):
    lb_removed = off_ball_remover(arg1, arg2)
    cb_removed = off_ball_remover(arg1, arg3)
    
    foo1 = {tuple(sublist) for sublist in lb_removed}
    foo2 = {tuple(sublist) for sublist in cb_removed}
    
    foo3 = foo1.intersection(foo2)
    
    final = [list(subset) for subset in foo3]
   
    
    return final

#print(defensive_locations(fronts.iloc[18,0], fronts.iloc[18,1], fronts.iloc[18,2]))

In [442]:
## This function will let us know if someone is lined up over center
def over_center(arg1, arg2, arg3):
    arg = defensive_locations(arg1, arg2, arg3)
    for item in arg:
        
        if 21<=item[1]<=23:
            return True
    return False
#over_center(fronts.iloc[2,0],fronts.iloc[2,1], fronts.iloc[2,2])

In [443]:
## This function will let us know if we have two defensive players lined up in a 3 technique
def three_technique(arg1, arg2, arg3):
    arg = defensive_locations(arg1, arg2, arg3)
    i=0
    for item in arg:
        if 15<=item[1]<=17:
            i+=1
        elif 27<=item[1]<=29:
            i+=1
    if i==2:
        
        return True
    else:
        
        return False
#three_technique(fronts.iloc[2,0],fronts.iloc[2,1],fronts.iloc[2,2])

In [444]:
## This will tell us if there is a rusher on the left side based on the positioning of the TEs
def left_edge_rusher(arg1, arg2, arg3, te_location):
    tes = tight_ends_number_and_location(te_location)
    d_line = defensive_locations(arg1, arg2, arg3)
    i = 0
    num_l_te = tes[0]
    if num_l_te == 0:
        for item in d_line:
            if item[1]>=31:
                i+=1
    elif num_l_te ==1:
        for item in d_line:
            if item[1]>=35:
                i+=1
    elif num_l_te ==2:
        for item in d_line:
            if item[1]>=39:
                i+=1
    elif num_l_te ==3:
        for item in d_line:
            if item[1]>=43:
                i+=1
    else:
        print("too many TE's")
    
    if i==1:
        return True
    else: 
        return False
#left_edge_rusher(fronts.iloc[2,0],fronts.iloc[2,1],fronts.iloc[2,2], fronts.iloc[2,-1])

In [445]:
## This will tell us if we have a rusher on the right side also based on the position of the TEs
def right_edge_rusher(arg1, arg2, arg3, te_location):
    tes = tight_ends_number_and_location(te_location)
    d_line = defensive_locations(arg1, arg2, arg3)
    i = 0
    num_r_te = tes[1]
    if num_r_te == 0:
        for item in d_line:
            if item[1]<=13:
                i+=1
    elif num_r_te ==1:
        for item in d_line:
            if item[1]<=9:
                i+=1
    elif num_r_te ==2:
        for item in d_line:
            if item[1]<=5:
                i+=1
    elif num_r_te ==3:
        for item in d_line:
            if item[1]<=1:
                i+=1
    else:
        print("too many TE's")
    
    if i==1:
        return True
    else: 
        return False
#right_edge_rusher(fronts.iloc[2,0],fronts.iloc[2,1],fronts.iloc[2,2], fronts.iloc[2,-1])

In [517]:
## This will take a row of a table and will classify and return TRUE if we have:
## 1. 2 D linemen in 3 technique
## 2. A left edge rusher
## 3. A right edge rusher
## 4. A center over the ball



def bear_jet_row_classifier(row):
    
    ## Arguments
    arg1 = row[0]
    arg2 = row[1]
    arg3 = row[2]
    te = row[-1]
    
    ## Checks to make
    three_tech = False
    le_rusher = False
    re_rusher = False
    over_center_rusher = False

    ## Check if there are 2 Dlinemen in 3 technique

    if three_technique(arg1, arg2, arg3) == True:
        three_tech = True
        
    else:
        
        three_tech = False

    ## Check if there is a left edge rusher

    if left_edge_rusher(arg1, arg2, arg3, te) == True:
        le_rusher = True
    else:
        le_rusher = False

    ## Check if there is a right edge rusher

    if right_edge_rusher(arg1, arg2, arg3, te) == True:
        re_rusher = True
    else:
        re_rusher = False

    ## Check if there is someone lined up over center

    if over_center(arg1, arg2, arg3) == True:
        over_center_rusher = True
    else: 
        over_center_rusher = False

    return three_tech, le_rusher, re_rusher, over_center_rusher

bear_jet_row_classifier(fronts.iloc[0,:])



/var/folders/8q/xfsr_zq50y9b99lq670pw6dc0000gn/T/ipykernel_8976/2955402979.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arg1 = row[0]
/var/folders/8q/xfsr_zq50y9b99lq670pw6dc0000gn/T/ipykernel_8976/2955402979.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arg2 = row[1]
/var/folders/8q/xfsr_zq50y9b99lq670pw6dc0000gn/T/ipykernel_8976/2955402979.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arg3 = row[2]
/var/folders/

(False, True, False, False)

In [498]:
## This will loop through all rows in a table
## and will create a list we can append as a column 
## to our table that has classified each row as a 'Bear'
## 'Jet' or 'Other' based on the 4 arguments classified by the above function 




def bear_jet_classifier(df):
    i=0
    fronts_list = []
    while i<len(df):
        row = df.iloc[i,:]
        inputs = bear_jet_row_classifier(row)
        if inputs[0] == True and inputs[1] == True and inputs[2]==True and inputs[3]==False:
            fronts_list.append("Jet")
        elif inputs[0] == True and inputs[1] == True and inputs[2]==True and inputs[3]==True:
            fronts_list.append("Bear")
        else:
            fronts_list.append("Other")
        i+=1
    return fronts_list
formations = bear_jet_classifier(fronts)

/var/folders/8q/xfsr_zq50y9b99lq670pw6dc0000gn/T/ipykernel_8976/214005012.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arg1 = row[0]
/var/folders/8q/xfsr_zq50y9b99lq670pw6dc0000gn/T/ipykernel_8976/214005012.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arg2 = row[1]
/var/folders/8q/xfsr_zq50y9b99lq670pw6dc0000gn/T/ipykernel_8976/214005012.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arg3 = row[2]
/var/folders/8q/

In [501]:
## Add the list of 'Bear', 'Jet', 'Other' classifications to the table as a new column 
fronts['Fronts'] = formations

In [506]:
fronts.head(2)

,pff_DEFFRONT,pff_LBDEPTH,pff_DBDEPTH,pff_OFFFORMATION,Reasons,Fronts
0,LOLB (08); LOLB (12); DLT (19); RILB (25); ROL...,LOLB (1); LOLB (0); RILB (1); ROLB (0),LCB (8); SCBL (6); FSL (14); SCBR (8); RCB (8),LWR; SLWR^; HB-L; SRWR^; RWR,Don't have two 3 technique DL,Other
1,LOLB (08); DLT (16); LILB (19); RILB (25); DRT...,LOLB (0); LILB (0); RILB (0); ROLB (0),LCB (6); SCBoL (3); SCBiL (10); FS (11); RCB (1),LWR^; HB-L; SRiWR; SRoWR^; RWR,2 Edge rushers 2 three technique DL but no one...,Jet


In [512]:
fronts.to_csv('Classified Jet-Bear Formations.csv', index=False, sep=',', quoting=csv.QUOTE_ALL)